In [13]:
# Suprimir warnings molestos
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Suprimir logs de transformers y pytorch_lightning
import logging
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)
logging.getLogger('transformers').setLevel(logging.ERROR)

# Recargar todos los módulos de AlignScore
import sys
import os

# Obtener el directorio actual del notebook
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

# Agregar el path al código fuente de AlignScore (path relativo)
alignscore_src_path = os.path.join(notebook_dir, 'AlignScore', 'src')
sys.path.insert(0, alignscore_src_path)

# Importar AlignScore
from alignscore import AlignScore

In [14]:
# Descargar el modelo AlignScore-base
# Primero necesitamos descargar el checkpoint del modelo
import os
import urllib.request

# Crear directorio para el checkpoint si no existe (path relativo)
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
checkpoint_dir = os.path.join(notebook_dir, 'AlignScore', 'checkpoints')
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'AlignScore-base.ckpt')

# Descargar el checkpoint si no existe
if not os.path.exists(checkpoint_path):
    url = "https://huggingface.co/yzha/AlignScore/resolve/main/AlignScore-base.ckpt" # AlignScore-large.ckpt"  # Para el modelo grande
    urllib.request.urlretrieve(url, checkpoint_path)
    print("Checkpoint descargado en:", checkpoint_path)
else:
    print("Checkpoint ya existe en:", checkpoint_path)

Checkpoint ya existe en: /Users/sbobi/Downloads/Uniandes 2025-02/NLP/Proyecto/Metricas LLM/AlignScore/checkpoints/AlignScore-base.ckpt


In [15]:
# Detectar el dispositivo disponible (GPU, MPS para Mac, o CPU)
import torch

def get_device():
    """
    Detecta y retorna el mejor dispositivo disponible.
    - cuda: Para GPUs NVIDIA
    - mps: Para GPUs Apple Silicon (M1, M2, M3, etc.)
    - cpu: Fallback si no hay GPU disponible
    """
    if torch.cuda.is_available():
        device = 'cuda'
        print(f"✓ GPU NVIDIA detectada: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print("✓ GPU Apple Silicon (MPS) detectada")
    else:
        device = 'cpu'
        print("⚠ No se detectó GPU, usando CPU")
    
    return device

device = get_device()
print(f"Dispositivo seleccionado: {device}")

✓ GPU Apple Silicon (MPS) detectada
Dispositivo seleccionado: mps


In [16]:
# Inicializar el scorer de AlignScore

scorer = AlignScore(
    model='roberta-base',
    batch_size=32,
    device=device,
    ckpt_path=checkpoint_path,
    evaluation_mode='nli_sp'
)

## Ejemplos del GitHub

In [17]:
# Ejemplo de consistencia factual
contexts = [
    "Children smiling and waving at camera.",
    "The NBA season of 1975-76 was the 30th season of the National Basketball Association."
]

claims = [
    "The kids are frowning.",  # Inconsistente
    "The 1975-76 season of the National Basketball Association was the 30th season of the NBA."  # Consistente
]

# Calcular scores
scores = scorer.score(contexts=contexts, claims=claims)

# Mostrar resultados
for i, (context, claim, score) in enumerate(zip(contexts, claims, scores)):
    print(f"\n--- Ejemplo {i+1} ---")
    print(f"Contexto: {context}")
    print(f"Afirmación: {claim}")
    print(f"AlignScore: {score:.4f}")
    print(f"Interpretación: {'Consistente' if score > 0.5 else 'Inconsistente'}")

Evaluating: 100%|██████████| 2/2 [00:00<00:00, 18.84it/s]


--- Ejemplo 1 ---
Contexto: Children smiling and waving at camera.
Afirmación: The kids are frowning.
AlignScore: 0.0003
Interpretación: Inconsistente

--- Ejemplo 2 ---
Contexto: The NBA season of 1975-76 was the 30th season of the National Basketball Association.
Afirmación: The 1975-76 season of the National Basketball Association was the 30th season of the NBA.
AlignScore: 0.9982
Interpretación: Consistente


## Datos Propios

In [24]:
# Múltiples ejemplos con diferentes niveles de consistencia
contexts_batch = [
    "Estoy usando una chuspa para llevar las compras del supermercado.",
]

claims_batch = [
    "Utilizo una bolsa de plástico para transportar las compras del súper.",
]

# Calcular todos los scores
scores_batch = scorer.score(contexts=contexts_batch, claims=claims_batch)

# Mostrar resultados en formato de tabla
import pandas as pd

df_results = pd.DataFrame({
    'Contexto': contexts_batch,
    'Afirmación': claims_batch,
    'AlignScore': [f"{s:.4f}" for s in scores_batch],
    'Consistencia': ['✓ Consistente' if s > 0.5 else '✗ Inconsistente' for s in scores_batch]
})

print(df_results.to_string(index=False))

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 12.78it/s]

                                                         Contexto                                                            Afirmación AlignScore  Consistencia
Estoy usando una chuspa para llevar las compras del supermercado. Utilizo una bolsa de plástico para transportar las compras del súper.     0.7884 ✓ Consistente
